In [2]:
# !source .venv/bin/activate
# !uv pip install -e .
from sqlalchemy import create_engine
from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

from esma.config.settings import settings

In [9]:
engine = create_engine(
    f"bigquery://{settings.gcp_project_id}/{settings.enaho_dataset}", 
    credentials_path=settings.google_application_credentials
)
db = SQLDatabase(engine)

/Users/carlostoruno/Documents/GitHub/equilibrium-sql-multiagent/.venv/lib/python3.13/site-packages/google/cloud/bigquery/client.py:613: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [10]:
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Enaho01_2024_100 LIMIT 5;")}')

Dialect: bigquery
Available tables: ['Enaho01A_2024_300', 'Enaho01A_2024_400', 'Enaho01A_2024_500', 'Enaho01_2024_100', 'Enaho01_2024_200', 'Enaho01_2024_601', 'Enaho01_2024_603', 'Enaho01_2024_604', 'Enaho01_2024_605', 'Enaho01_2024_609', 'Enaho01_2024_612']
Sample output: [(2024, 1, 15024, 11, 11, 4, 10101, 4, 1, 1, 20240116, 7, ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' 

In [11]:
llm = init_chat_model(
    settings.default_model, 
    temperature=settings.temperature, 
    max_tokens=settings.max_tokens
)

In [12]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!

